In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [2]:
from sklearn.preprocessing import LabelEncoder,LabelBinarizer,MinMaxScaler,OneHotEncoder

In [39]:
act_train_data = pd.read_csv("data/act_train.csv",dtype={'people_id': np.str, 'activity_id': np.str, 'outcome': np.int8}, parse_dates=['date'])
act_test_data  = pd.read_csv("data/act_test.csv", dtype={'people_id': np.str, 'activity_id': np.str}, parse_dates=['date'])

In [4]:
people_data    = pd.read_csv("data/people.csv", dtype={'people_id': np.str, 'activity_id': np.str, 'char_38': np.int32}, parse_dates=['date'])

In [5]:
# people_data['date'].dt.year

In [6]:
act_train_data = act_train_data.drop('char_10', axis = 1)
act_test_data=act_test_data.drop('char_10',axis=1)

In [7]:
y_train = act_train_data.outcome
act_train_data = act_train_data.drop('outcome',axis=1)

In [40]:
train_num = act_train_data.shape[0]
test_activity_id = act_test_data['activity_id'].values

In [41]:
test_activity_id

array(['act1_249281', 'act2_230855', 'act1_240724', ..., 'act2_448830',
       'act2_450133', 'act2_847967'], dtype=object)

In [9]:
act_train_data.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data = pd.concat([act_train_data, act_test_data])

In [11]:
data = pd.merge(data, people_data, how='left',on='people_id').fillna('missing')

In [12]:
del act_train_data
del act_test_data
del people_data

In [13]:
data['year_x'] = data['date_x'].dt.year
data['month_x'] = data['date_x'].dt.month
data['day_x'] = data['date_x'].dt.day
data['isWeekend_x'] = data['date_x'].dt.weekday >= 5

In [14]:
data['year_y'] = data['date_y'].dt.year
data['month_y'] = data['date_y'].dt.month
data['day_y'] = data['date_y'].dt.day
data['isWeekend_y'] = data['date_y'].dt.weekday >= 5

In [15]:
# data['isWeekend_y']

In [16]:
data = data.drop('date_x', axis = 1)

In [17]:
data = data.drop('date_y', axis = 1)

In [18]:
data.columns

Index([u'people_id', u'activity_id', u'activity_category', u'char_1_x',
       u'char_2_x', u'char_3_x', u'char_4_x', u'char_5_x', u'char_6_x',
       u'char_7_x', u'char_8_x', u'char_9_x', u'char_1_y', u'group_1',
       u'char_2_y', u'char_3_y', u'char_4_y', u'char_5_y', u'char_6_y',
       u'char_7_y', u'char_8_y', u'char_9_y', u'char_10', u'char_11',
       u'char_12', u'char_13', u'char_14', u'char_15', u'char_16', u'char_17',
       u'char_18', u'char_19', u'char_20', u'char_21', u'char_22', u'char_23',
       u'char_24', u'char_25', u'char_26', u'char_27', u'char_28', u'char_29',
       u'char_30', u'char_31', u'char_32', u'char_33', u'char_34', u'char_35',
       u'char_36', u'char_37', u'char_38', u'year_x', u'month_x', u'day_x',
       u'isWeekend_x', u'year_y', u'month_y', u'day_y', u'isWeekend_y'],
      dtype='object')

In [19]:
data.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_37,char_38,year_x,month_x,day_x,isWeekend_x,year_y,month_y,day_y,isWeekend_y
0,ppl_100,act2_1734928,type 4,missing,missing,missing,missing,missing,missing,missing,...,False,36,2023,8,26,True,2021,6,29,False
1,ppl_100,act2_2434093,type 2,missing,missing,missing,missing,missing,missing,missing,...,False,36,2022,9,27,False,2021,6,29,False
2,ppl_100,act2_3404049,type 2,missing,missing,missing,missing,missing,missing,missing,...,False,36,2022,9,27,False,2021,6,29,False
3,ppl_100,act2_3651215,type 2,missing,missing,missing,missing,missing,missing,missing,...,False,36,2023,8,4,False,2021,6,29,False
4,ppl_100,act2_4109017,type 2,missing,missing,missing,missing,missing,missing,missing,...,False,36,2023,8,26,True,2021,6,29,False


In [20]:
data = data.drop('activity_id', axis = 1)
data = data.drop('people_id', axis = 1)

In [21]:
columns = data.columns.tolist()

In [22]:
columns.remove('char_38')

In [23]:
for c in columns:
    data[c] = LabelEncoder().fit_transform(data[c].values)

In [24]:
data.head()

,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,...,char_37,char_38,year_x,month_x,day_x,isWeekend_x,year_y,month_y,day_y,isWeekend_y
0,3,0,0,0,0,0,0,0,0,0,...,0,36,1,7,25,1,1,5,28,0
1,1,0,0,0,0,0,0,0,0,0,...,0,36,0,8,26,0,1,5,28,0
2,1,0,0,0,0,0,0,0,0,0,...,0,36,0,8,26,0,1,5,28,0
3,1,0,0,0,0,0,0,0,0,0,...,0,36,1,7,3,0,1,5,28,0
4,1,0,0,0,0,0,0,0,0,0,...,0,36,1,7,25,1,1,5,28,0


In [25]:
categorical=['group_1','activity_category','char_1_x','char_2_x','char_3_x','char_4_x','char_5_x','char_6_x','char_7_x','char_8_x','char_9_x','char_2_y','char_3_y','char_4_y','char_5_y','char_6_y','char_7_y','char_8_y','char_9_y']
not_categorical=[]
for category in data.columns:
    if category not in categorical:
        not_categorical.append(category)

In [26]:
enc = OneHotEncoder(handle_unknown='ignore')
enc = enc.fit(data[categorical])

In [27]:
train_cat_sparse = enc.transform(data[:train_num][categorical])
test_cat_sparse = enc.transform(data[train_num :][categorical])

In [28]:
test_cat_sparse.shape

(498687, 34527)

In [29]:
from scipy.sparse import hstack
train_sparse = hstack((data[:train_num][not_categorical],  train_cat_sparse))
test_sparse = hstack((data[train_num :][not_categorical], test_cat_sparse))

In [30]:
train_sparse.shape

(2197291, 34565)

In [31]:
del data

In [32]:
y_train

0          0
1          0
2          0
3          0
4          0
5          0
6          1
7          1
8          1
9          1
10         1
11         1
12         1
13         1
14         1
15         1
16         1
17         1
18         1
19         1
20         1
21         1
22         1
23         1
24         1
25         1
26         1
27         1
28         1
29         1
          ..
2197261    1
2197262    1
2197263    1
2197264    1
2197265    1
2197266    1
2197267    1
2197268    1
2197269    1
2197270    1
2197271    1
2197272    1
2197273    1
2197274    1
2197275    1
2197276    1
2197277    1
2197278    1
2197279    1
2197280    1
2197281    1
2197282    1
2197283    1
2197284    1
2197285    1
2197286    1
2197287    1
2197288    1
2197289    1
2197290    1
Name: outcome, dtype: int8

In [33]:
dtrain = xgb.DMatrix(train_sparse, label=y_train)

In [34]:
dtest = xgb.DMatrix(test_sparse)

In [35]:
del train_sparse
del test_sparse

In [36]:
param = {'max_depth':10, 'eta':0.02, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
param['subsample'] = 0.7
param['colsample_bytree']= 0.7
param['min_child_weight'] = 0
param['booster'] = "gblinear"

In [37]:
watchlist  = [(dtrain,'train')]
num_round = 300
early_stopping_rounds=10
bst = xgb.train(param, dtrain, num_round, watchlist,early_stopping_rounds=early_stopping_rounds)

[0]	train-auc:0.887743
Will train until train-auc hasn't improved in 10 rounds.
[1]	train-auc:0.895468
[2]	train-auc:0.903455
[3]	train-auc:0.911746
[4]	train-auc:0.91998
[5]	train-auc:0.927868
[6]	train-auc:0.935187
[7]	train-auc:0.941903
[8]	train-auc:0.948173
[9]	train-auc:0.953991
[10]	train-auc:0.959337
[11]	train-auc:0.964184
[12]	train-auc:0.968493
[13]	train-auc:0.972264
[14]	train-auc:0.975512
[15]	train-auc:0.978281
[16]	train-auc:0.980623
[17]	train-auc:0.982597
[18]	train-auc:0.984245
[19]	train-auc:0.985618
[20]	train-auc:0.98677
[21]	train-auc:0.987743
[22]	train-auc:0.988564
[23]	train-auc:0.989268
[24]	train-auc:0.989879
[25]	train-auc:0.990414
[26]	train-auc:0.990889
[27]	train-auc:0.991313
[28]	train-auc:0.991695
[29]	train-auc:0.992041
[30]	train-auc:0.992355
[31]	train-auc:0.992643
[32]	train-auc:0.992908
[33]	train-auc:0.993151
[34]	train-auc:0.993375
[35]	train-auc:0.993582
[36]	train-auc:0.993774
[37]	train-auc:0.993952
[38]	train-auc:0.994119
[39]	train-auc:0.99

In [42]:
ypred = bst.predict(dtest)
output = pd.DataFrame({ 'activity_id' : test_activity_id, 'outcome': ypred })
output.head()

,activity_id,outcome
0,act1_249281,0.001425
1,act2_230855,0.001529
2,act1_240724,0.999439
3,act1_83552,0.999740
4,act2_1043301,0.999447
